In [16]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import pandas as pd
import datetime as dt

#Where to save compiled csv file
local_folder = "C:/Data/SDCovid/Dailies/"
#location of our data
folder_url = 'http://shulok.com/SD_Covid_data/'

#get the contents of the folder
url = requests.get(folder_url).text
soup = BeautifulSoup(url)
print(soup)


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<html>
<head>
<title>Index of /SD_Covid_data</title>
</head>
<body>
<h1>Index of /SD_Covid_data</h1>
<ul><li><a href="/"> Parent Directory</a></li>
<li><a href="tabula-COVID-19%20Deaths%20by%20Demographics%204-10-2020.csv"> tabula-COVID-19 Deaths by Demographics 4-10-2020.csv</a></li>
<li><a href="tabula-COVID-19%20Deaths%20by%20Demographics%204-11-2020.csv"> tabula-COVID-19 Deaths by Demographics 4-11-2020.csv</a></li>
<li><a href="tabula-COVID-19%20Deaths%20by%20Demographics%204-13-2020.csv"> tabula-COVID-19 Deaths by Demographics 4-13-2020.csv</a></li>
<li><a href="tabula-COVID-19%20Deaths%20by%20Demographics%204-17-2020.csv"> tabula-COVID-19 Deaths by Demographics 4-17-2020.csv</a></li>
<li><a href="tabula-COVID-19%20Deaths%20by%20Demographics%204-20-2020.csv"> tabula-COVID-19 Deaths by Demographics 4-20-2020.csv</a></li>
<li><a href="tabula-COVID-19%20Deaths%20by%20Demographics%204-22-2020.csv"> tabula-COVID-19 Deaths by Demo

In [17]:
frames = []
dateformat = "%m-%d-%Y"
last_date = None
switch_date = dt.datetime(2020,6,25)
skip_nrows = 1
for link in soup.findAll("a"):
    current_link = link.get("href")
    if current_link.endswith('csv') and current_link.count('Ethnicity') == 1:

        #get file date which is right before the file extension
        mylist = current_link.split("%20")
        datestr = mylist[5].split('.')[0]
        
        #convert date string to date object
        date = dt.datetime.strptime(datestr, dateformat)
        if last_date == None or last_date < date:
            last_date = date
            
        if date > switch_date:
            skip_nrows = 2
            
        # Read the source file, but use the first column as an index
        df = pd.read_csv(folder_url + current_link, skiprows=skip_nrows)
        
        #The file date is the date when the data was updated
        #It includes data up to the day before
        df['Date'] = date - dt.timedelta(days=1)
        df.columns = ['Race and Ethnicity', 'Count', '% with known Race/Ethnicity', 'per 100k', 'Date']
       
        frames.append(df)
    
df = pd.concat(frames, sort=True) # Concatenate all of the transposed frames

#reorder the columns
df=df.reindex(columns= ['Date', 'Race and Ethnicity', 'Count', '% with known Race/Ethnicity', 'per 100k'])

df.sort_values(by=['Date'], inplace=True, ascending=True)

filename = 'COVID_19_Race_and_Ethnicity_Statistics_San_Diego_County_%s.csv' % last_date.strftime(dateformat)
print(filename)

df.to_csv(local_folder + filename, index=False)# The index column is useless, so drop it. 
df.head()


COVID_19_Race_and_Ethnicity_Statistics_San_Diego_County_07-21-2020.csv


,Date,Race and Ethnicity,Count,% with known Race/Ethnicity,per 100k
0,2020-06-25,Hispanic or Latino,"6,665",65.8%,579.2
1,2020-06-25,White,"2,224",21.9%,145.8
2,2020-06-25,Black or African American,416,4.1%,281.6
3,2020-06-25,Asian,626,6.2%,171.9
4,2020-06-25,Native Hawaiian/Pacific Islander,92,0.9%,625.3
